In [1]:
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Buscamos que la url sea como nosotros la pedimos

date_1 = '2024-10-25'
date_2 = '2024-10-27'
ciudad = 'berlin'
pagina = 1

url = f"https://www.civitatis.com/es/{ciudad}/?page={pagina}&fromDate={date_1}&toDate={date_2}"

In [4]:
#Abrimos en navegador con la Url introducida
driver = webdriver.Chrome()
url_wunder = url

driver.get(url_wunder)
driver.maximize_window()
sleep(5)
driver.close()

In [5]:
url_civitatis = "https://www.civitatis.com/es/{ciudad}/?page={pagina}&fromDate={date_1}&toDate={date_2}"
res_civitatis = requests.get(url)
print(f"La respuesta de la petición es: {res_civitatis.status_code}")

La respuesta de la petición es: 200


In [6]:
#creamos nuestra sopa 

sopa_civitatis = BeautifulSoup(res_civitatis.content, "html.parser")

In [7]:
# Creamos nuestro bucle para iterar de los datos scrapeados y coger lo que no shace falta

nombres_actividades = []
descripciones_actividades = []
precios_actividades = []

bloque_total = sopa_civitatis.findAll("div", {"class": "o-search-list__item"} )
for i in bloque_total:
    nombre_actividad = i.find("h2").text.strip()
    descripcion_actividad = i.find("div",{"class":"comfort-card__text l-list-card__text"}).text.strip()
    precio_actividad = i.find("span",{"class":"comfort-card__price__text"}).text.strip()

# Lo añadimos a la lista de listas
    nombres_actividades.append(nombre_actividad)
    descripciones_actividades.append(descripcion_actividad)
    precios_actividades.append(precio_actividad)

# Creamos un documento
datos = {
    'Nombre Actividad': nombres_actividades,
    'Descripción Actividad': descripciones_actividades,
    'Precio Actividad': precios_actividades
}

# Creamos los datos de diccionario en un DataFrame
df_berlin = pd.DataFrame(datos)
df_berlin

,Nombre Actividad,Descripción Actividad,Precio Actividad
0,Campo de concentración de Sachsenhausen,"Dura, pero imprescindible. La excursión al cam...",30 €
1,Free tour por Berlín,"Si viajáis a la capital de Alemania, no dudéis...",¡Gratis!
2,Excursión a Potsdam y Sachsenhausen,En esta excursión a Potsdam y Sachsenhausen vi...,65 €
3,Excursión a Potsdam,¿Queréis conocer una de las ciudades más bella...,26 €
4,Tour de Berlín al completo,El tour de Berlín al completo es perfecto para...,40 €
5,Visita guiada por el Berlín alternativo,En esta visita guiada por el Berlín alternativ...,20 €
6,Traslados en Berlín,"Servicio privado, puerta a puerta y al mejor p...",62 €
7,Free tour del Muro de Berlín y la Guerra Fría,En este free tour del Muro de Berlín y la Guer...,¡Gratis!
8,Paseo histórico por el Berlín del nazismo,En este paseo histórico por Berlín nos acercar...,22 €
9,Visita guiada por Berlín,No hay mejor forma de empezar un viaje que con...,22 €


In [10]:
#Guardamos el DF como CSV
df_berlin.to_csv("../data/ActividadesBerlin.csv")